In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import time
import datetime

# from bs4 import BeautifulSoup
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service as ChromeService
# from selenium.webdriver.common.by import By
# from webdriver_manager.chrome import ChromeDriverManager

import pprint
import requests

from tqdm import tqdm

In [13]:
cd '/content/drive/MyDrive/university/Year 3 Term 2/Data Science/Project/Project_2'

/content/drive/.shortcut-targets-by-id/11daT7KKDj0zZzXrrhzIm8Lmfu5LugfH1/university/Year 3 Term 2/Data Science/Project/Project_2


# **Functions**

In [ ]:
def provinceTH_to_provinceEN(dataframe_mapper, provinceTH, selectedProvince):
    try:
        spec_row = dataframe_mapper[dataframe_mapper['ProvinceTH'] == provinceTH]
        provinceEN = spec_row['ProvinceEN'].values[0]
        return provinceEN
    except:
        return selectedProvince
def getProvinceFromFolderName(filePath, config):
    list_of_cut_file_path = filePath.split('/')
    #have folder of province
    province_from_folder = None
    if len(list_of_cut_file_path) >= 6:
        folder_name = list_of_cut_file_path[4]
        if folder_name not in config['vagueProvince']:
            province_from_folder = refineProvince(folder_name)
    return province_from_folder

def getPayload(url): 
    print(f'url : {url}')
    response = requests.get(url)
    print(f"request return {response.status_code} : '{response.reason}'")
    payload = response.json()
    return payload

def refineProvince(province):
    if province is None:
        return None
    if province == 'กรุงเทพมหานคร':
        return province
    else:
        return f'จ.{province}'

def compareProvince(province_from_keyword, province_from_folder_name):
  ###### fodler name has higher priority compared to from keyword
  if province_from_folder_name is None:
    return province_from_keyword
  return province_from_folder_name

def refinePlusGetBothKeywordProvince(word):
    if word.find('ปิดสถานี') != -1:
        reversed_word = word[::-1] ### in case there is too many () and we know that (ปิดสถานี) is the last so we reverse to find the index of () easier
        start_index_in_parenthesis = (len(word) - 1) - reversed_word.find('(')
        end_index_in_parenthesis = (len(word) - 1) - reversed_word.find(')')
        if (end_index_in_parenthesis - 1)  - start_index_in_parenthesis > 8:
        ######## first case (finding word inside parenthesis alongside with ปิดสถานี) เช่น นครสวรรค์(วิทยาลัยอาชีวศึกษา_ปิดสถานี)
          if word[start_index_in_parenthesis:end_index_in_parenthesis].find('_') != -1:
            print(1)
            list_of_string_of_cut_word = word.split("_")
            first_case_keyword = list_of_string_of_cut_word[0][start_index_in_parenthesis + 1 :].strip()
            province = word[:start_index_in_parenthesis]
            return first_case_keyword, province
        ######### second case เช่น มหาวิทยาลัยราชภัฏจันทรเกษม(ปิดสถานี 1 ธ.ค.58)
          else:
            print(2)
            second_case_keyword = word[:start_index_in_parenthesis]
            province = None
            return second_case_keyword, province
        else:  
            list_of_string_of_cut_word = word.split('_')
        ######### third case เช่น สนามกีฬากลางองค์การบริหารส่วนจังหวัดลำพูน(ปิดสถานี).xlsx
            if len(list_of_string_of_cut_word) == 1:
                print(3)
                province = None
                third_case_keyword = word[: start_index_in_parenthesis].strip()
                return third_case_keyword, province
       ######### fourth case เช่น ศูนย์วิศวกรรมการแพทย์ที่ 1_ราชบุรี (ปิดสถานี)
            else:
                print(4)
                fourth_case_keyword, province = list_of_string_of_cut_word[0].strip(), list_of_string_of_cut_word[1][:start_index_in_parenthesis].strip()
                return fourth_case_keyword, province

    else:
      ######### fifth case มีคำว่า จ. ในคำ เช่น โรงเรียนอนุบาลศรีอรัญโญทัย จ.สระแก้ว
        abbreviation_word_index = word.find('จ.')
        full_word_index = word.find('จังหวัด')
        if abbreviation_word_index != -1:
            print(5)
            fifth_case_keyword = word[:abbreviation_word_index].strip()
            province = None
            if word.find('อบจ.') == -1:
                province = word[abbreviation_word_index+2:].strip()
            return fifth_case_keyword, province
        if full_word_index != -1:
      ########## seventh case มีคำว่า จังหวัดในคำ เช่น อุตุนิยมวิทยาจังหวัดลำพูน
            print(6)
            sixth_case_keyword = word[:full_word_index].strip()
            province = word[full_word_index+len('จังหวัด'):].strip()
            return sixth_case_keyword, province
    #     else:
    #   ########### seventh case เช่น ริมถนนกาญจนาภิเษก เขตบางขุนเทียน
    #       roadside_index = word.find('ริม')
    #       if roadside_index != -1:
    #         print(7)
    #         seventh_case_keyword = word.split()[0]
    #         seventh_case_keyword = seventh_case_keyword[roadside_index+3:]
    #         province = None
    #         return seventh_case_keyword, province
        return word.strip(), None

def checkStatusStation(stationName):
    findIndex = stationName.find('ปิดสถานี')
    isOpenStation = False
    if findIndex == -1:
        isOpenStation = True
    return isOpenStation

def clickSearchButton(driver):
    search_button = driver.find_element(By.XPATH, "//button[@id='searchbox-searchbutton']")
    search_button.click()
    
def scrapeLatLongBySelenium(driver, keyword):
    ######### searchElement.send_keys(keyword)
    searchElement.clear()
    searchElement.send_keys(keyword)
    clickSearchButton(driver)

    ##### wait for searching location to finish
    # waitSearchingLocation(driver, "//div[@class='nhb85d FOKsab xoLGzf Hk4XGb AwQxhf']")
    time.sleep(8)
    ##### check if keyword is specific enough (if enough -> there wouldn't be a choice to choose -> check_exist == false)
    # if check_exists_by_xpath(driver, "//div[@jsan='t-dgE5uNmzjiE,7.m6QErb,7.WNBkOb,0.role']"):
    #     #### pick the first one
    # #     text = driver.find_element(By.XPATH,"//div[@jsan='t-dgE5uNmzjiE,7.m6QErb,7.WNBkOb,0.role']")
    # #     print(text.text)
    # #     all_link = driver.find_elements_by_css_selector('.778 a')
    # #     for link in all_link:
    # #         print(link.text, link.get_attribute('href'))
    # #         print('----')
    #     counter += 1
    #     print(counter)
    url = driver.current_url
    cutWordIdx = url.find('@') + 1 ##not include @
    list_of_lat_long = url[cutWordIdx:].split(',')
    lat, long = list_of_lat_long[:2]      
    return lat, long

def check_exists_by_xpath(driver, xpath):
    try:
        driver.find_element(By.XPATH, xpath)
        return True
    except:
        return False

def waitSearchingLocation(driver, xpath):
    while True:
        if check_exists_by_xpath(driver, xpath) == True:
            return

# **Preprocess the SFA data**

In [ ]:
sfa_data_url = './data/SFA Data /ID_DATA_SFA.csv'
sfa_info_url = './data/SFA Data /ID_INFO_SFA.csv'
mapperDataframeUrl = './outsource_data/mapper-provinces-thailand.xlsx'

In [ ]:
###### read file csv, excel
sfa_data = pd.read_csv(sfa_data_url)
sfa_info = pd.read_csv(sfa_info_url)
mapper_dataframe = pd.read_excel(mapperDataframeUrl)

####### preprocess the cs, excel file
sfa_info.dropna(inplace=True)
sfa_info['lat'] = sfa_info['lat'].astype(str)
sfa_info['long'] = sfa_info['long'].astype(str)

sfa_data.drop(columns = ['Unnamed: 0'], inplace=True)
sfa_info.drop(columns = ['id'], inplace=True)

mapper_dataframe['ProvinceEN'] = mapper_dataframe['ProvinceEN'].str.replace(' Province','')
mapper_dataframe['ProvinceTH'] = mapper_dataframe['ProvinceTH'].str.replace('จังหวัด','จ.')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
mapper_dataframe.to_csv('./outsource_data/mapper-provinces-thailand.csv')

## **fetch data from using only latitude, longtitude**

In [ ]:
################################## FETCH lat long #########################################
##################################
#config
config = {
    'apiKey': 'b42b79d9e2205bc45e6d79113e8d2db8',
}
sfa_dictionary = []
######### iterate each row
for index, row in tqdm(sfa_info.iterrows()):
    url = f"https://api.longdo.com/map/services/address?lon={row['long']}&lat={row['lat']}&noelevation=1&key={config['apiKey']}"

    ##request and response
    response = requests.get(url)
    payload = response.json()
    #####add new data in dataframe
    provinceTH = payload['province']
    provinceEN = provinceTH_to_provinceEN(mapper_dataframe, provinceTH)
    newDic = [row['device_id'], None, row['name_en'], row['lat'], row['long'], provinceEN, None]
    sfa_dictionary.append(newDic)
    ############## wait for the limit count of api in certain time 
    if index%60 == 0:
      time.sleep(5)

sfa_dictionary = pd.DataFrame(sfa_dictionary, columns = ['device_id','stationId', 'name', 'lat', 'long', 'province', 'isStationOpen'])
sfa_dictionary.to_csv('./processed_data/SFA_DICTIONARY.csv')


444it [06:44,  1.10it/s]


## **รวมไฟล์ของทั้ง data กับ info**

In [ ]:
sfa_merge = pd.merge(sfa_data, sfa_dictionary)
sfa_merge.drop(columns = ['stationId','isStationOpen'],inplace=True)

In [ ]:
sfa_merge.to_csv('./processed_data/SFA_MERGE.csv')

# **fetch data from keyword, lat, long to create PCD dictionary**

In [ ]:
dict_url = './processed_data/PCD_Data_Before_Stations.csv'
dictionary = pd.read_csv(dict_url)

In [ ]:
config = {
    'apiKey': 'b42b79d9e2205bc45e6d79113e8d2db8',
    'vagueProvince' : ['ambient', 'roadside'],
    'mapperDataframeUrl' : './outsource_data/mapper-provinces-thailand.csv'
}

mapper_dataframe = pd.read_csv(config['mapperDataframeUrl'])

PCD_DICTIONARY = []
##### open chrome ##########
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

#####open website
urlWeb = 'https://www.google.co.th/maps'
driver.get(url=urlWeb)
searchElement = driver.find_element(By.XPATH, "//input[@id='searchboxinput']")

for idx, row in dictionary.iterrows():
    ########### extract features
    print(f"word is {row['stationName']}")
    province_from_folder = getProvinceFromFolderName(row['filePath'], config)
    keyword, province_from_keyword = refinePlusGetBothKeywordProvince(row['stationName'])
    province_from_keyword = refineProvince(province_from_keyword)
    selectedProvince = compareProvince(province_from_keyword, province_from_folder)
    print(f"keyword is {keyword}")
    print(f'province from keyword and folder is {province_from_keyword} , {province_from_folder}')
    if selectedProvince is not None:
        keyword += selectedProvince

    ############ fetch first url(2nd tries)
    lat, long = scrapeLatLongBySelenium(driver, keyword)
    print(lat, long)
    ######fetch url from lat, long we have to get the province we need
    url = f"https://api.longdo.com/map/services/address?lon={long}&lat={lat}&noelevation=1&key={config['apiKey']}"
    ##request and response
    response = requests.get(url)
    payload = response.json()
    if 'province' not in payload:
        provinceEN = None
    else:
        provinceTH = payload['province']
        provinceEN = provinceTH_to_provinceEN(mapper_dataframe, provinceTH, selectedProvince)

    #####add new data in dataframe
    isOpenStation = checkStatusStation(row['stationName'])
    newDic = [row['stationID'], keyword, lat, long, provinceEN, isOpenStation]
    print()
    PCD_DICTIONARY.append(newDic)

############## wait for the limit count of api in certain time 
    if idx%30 == 0:
        time.sleep(5)




====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\aong1\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


1
word is มหาวิทยาลัยราชภัฏบ้านสมเด็จเจ้าพระยา
['02t', 'มหาวิทยาลัยราชภัฏบ้านสมเด็จเจ้าพระยา', '13.7322228', '100.4881036', 'Bangkok', True]


word is ริมถนนกาญจนาภิเษก เขตบางขุนเทียน
['03t', 'ริมถนนกาญจนาภิเษก เขตบางขุนเทียน', '13.7840513', '100.3894034', 'Bangkok', True]


word is ไปรษณีย์ราษฎร์บูรณะ (ปิดสถานี)
3
['03t_1', 'ไปรษณีย์ราษฎร์บูรณะ', '13.6760729', '100.4976317', 'Bangkok', False]


word is หมวดการทางบางขุนเทียนที่ 2(ปิดสถานี)
3
['03t_2', 'หมวดการทางบางขุนเทียนที่ 2', '13.6576556', '100.426853', 'Bangkok', False]


word is กรมอุตุนิยมวิทยาบางนา
['05t', 'กรมอุตุนิยมวิทยาบางนา', '13.6675215', '100.6028436', 'Bangkok', True]


word is มหาวิทยาลัยราชภัฏจันทรเกษม(ปิดสถานี 1 ธ.ค.58)
2
['07t', 'มหาวิทยาลัยราชภัฏจันทรเกษม', '13.8188261', '100.575115', 'Bangkok', False]


word is ศูนย์ฟื้นฟูอาชีพคนพิการและทุพพลภาพพระประแดง
['08t', 'ศูนย์ฟื้นฟูอาชีพคนพิการและทุพพลภาพพระประแดงจ.สมุทรปราการ', '13.6628142', '100.5348939', 'Samut Prakan', True]


word is เคหะชุมชนคลองจั่น
['10t', 'เคหะช

In [ ]:
PCD_DICTIONARY = pd.DataFrame(PCD_DICTIONARY, columns = ['stationId', 'name', 'lat', 'long', 'province', 'isStationOpen'])
PCD_DICTIONARY.to_csv('./processed_data/PCD_DICTIONARY.csv')